<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b4a7d8ecc83930ef9c9fd6b8eb8950d83145aa79e950da4bd12a4ffe2a4da228
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 09:54:48
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -24.46 K (-0.17%)
Current PnL: -27.00 L (-17.44%)
CY Booked + Current PnL: -12.71 L (-8.21%)
-------------------
Total profit:  1.39 L
Total loss:  -28.38 L
-------------------
Total Booked + Current PnL: 14.27 L (11.17%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.63 L (65.42%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.08%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.03,-18.05,22.14,0.09,18284.0,-18195.0,82582.0,91.68,35.0,M-SC,2.48,253.0,-1.00,0.59,8.50,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.39,8.65,10.17,19.69,21186.0,16578.0,208320.0,-1.91,57.0,X-MC,2.56,78.0,0.78,1.49,16.03,XY25,NTT,AC
50,MASFIN,398.61,-0.08,-2.86,25.65,22.05,24412.0,-2805.0,95175.0,-16.44,52.0,H-SC,6.90,164.0,-0.11,0.68,38.34,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.24,-14.47,16.94,0.02,25291.0,-25263.0,149296.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.07,29.86,OX40N,NTT,PAINTS
43,ITC,452.00,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.70,3.61,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.1,1.55,-34.72,113.87,39.61,76190.0,-35590.0,66910.0,-55.84,37.0,H-SC,29.32,135.0,-0.47,0.48,5.42,AR,ATH,MISC
13,BSOFT,831.7,1.35,-21.71,92.59,50.79,101962.0,-30531.0,110122.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.22,XR,ATH,IT
63,SFL,1287.0,1.31,-43.57,124.39,26.63,184067.0,-114242.0,147976.0,12.11,31.0,M-SC,11.86,239.0,-0.62,1.06,1.31,XY24,NTT,MISC
75,TMPV,600.0,0.97,-28.51,72.02,22.98,116308.0,-64404.0,161494.0,-25.52,35.0,X-LC,6.72,3.0,-0.55,1.16,1.57,XY24,NTT,AUTO
16,CERA,9475.0,0.83,-29.43,75.56,23.89,93793.0,-51772.0,124131.0,-31.98,43.0,H-SC,10.77,157.0,-0.55,0.89,7.20,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-1.96,-16.19,75.34,46.95,79113.0,-20289.0,105008.0,-22.59,41.0,H-SC,7.99,154.0,-0.26,0.75,31.55,MH,ATH,POWER
18,COFFEEDAY,80.00,-1.80,-41.85,132.63,35.27,87574.0,-47520.0,66029.0,-55.73,45.0,L-SC,10.65,270.0,-0.54,0.47,61.61,XR,NTT,HOTELS
28,GREENPANEL,537.00,-1.71,-38.58,133.06,43.14,149613.0,-70638.0,112440.0,151.94,30.0,M-SC,9.99,240.0,-0.47,0.81,5.77,XY24,NTT,MISC
71,TANLA,1943.92,-1.70,-36.78,247.50,119.69,423661.0,-99585.0,171176.0,-36.34,43.0,H-SC,7.95,156.0,-0.24,1.23,35.23,AR,ATH,IT
19,COLPAL,3726.84,-1.48,-19.18,75.09,41.51,159829.0,-50515.0,212850.0,-3.66,41.0,X-MC,7.30,61.0,-0.32,1.53,1.27,XY25,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.76,-2.01,112.17,107.91,162290.0,-2962.0,144682.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.04,7.88,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.24,-14.47,16.94,0.02,25291.0,-25263.0,149296.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.07,29.86,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.03,-18.05,22.14,0.09,18284.0,-18195.0,82582.0,91.68,35.0,M-SC,2.48,253.0,-1.00,0.59,8.50,OX40N,NTT,DURABLES
66,SIS,528.00,-0.00,-24.18,59.93,21.26,50454.0,-26844.0,84188.0,1986.01,50.0,H-SC,5.06,166.0,-0.53,0.60,13.75,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.49,-25.44,52.19,13.47,104938.0,-68598.0,201069.0,-69.28,34.0,H-SC,5.65,158.0,-0.65,1.44,2.24,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.64,7.0,61.0,72.27,110596.0,11857.0,181305.0,-8.35,65.0,M-LC,2.61,99.0,0.11,1.3,13.6,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.64,7.00,61.00,72.27,110596.0,11857.0,181305.0,-8.35,65.0,M-LC,2.61,99.0,0.11,1.30,13.60,XR,NTT,IT
25,FINCABLES,1641.55,-0.76,-2.01,112.17,107.91,162290.0,-2962.0,144682.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.04,7.88,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.79,-3.79,114.86,106.72,136301.0,-4669.0,118667.0,-52.34,37.0,H-SC,3.28,139.0,-0.03,0.85,18.71,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.33,-3.26,40.20,35.63,81503.0,-6829.0,202745.0,-16.74,38.0,H-MC,2.85,119.0,-0.08,1.45,12.87,AR,ATH,PHARMA
37,IEX,219.00,0.26,-4.47,55.58,48.63,106164.0,-8943.0,191011.0,-36.48,47.0,H-SC,14.08,136.0,-0.08,1.37,6.56,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.72,-18.84,91.56,55.47,180793.0,-45827.0,197459.0,-28.03,23.0,X-MC,13.42,64.0,-0.25,1.42,1.17,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.33,-10.05,42.42,28.11,76854.0,-20235.0,181175.0,-34.87,27.0,X-MC,13.19,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.98,-39.39,123.07,35.20,95979.0,-50683.0,77987.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.97,-23.81,101.45,53.50,233250.0,-71835.0,229916.0,-63.12,29.0,X-MC,3.28,58.0,-0.31,1.65,2.12,XY24,NTT,FMCG
33,HINDUNILVR,2922.00,0.01,-11.26,28.08,13.67,64059.0,-28933.0,228130.0,-17.95,32.0,X-LC,2.30,14.0,-0.45,1.64,7.70,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.07,-1.17,25.94,24.48,64570.0,-2937.0,248919.0,-81.09,44.0,X-SC,0.27,86.0,-0.05,1.79,21.70,X40N,NTT,MISC
43,ITC,452.0,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.70,3.61,X40,NTT,FMCG
20,DABUR,735.0,-0.04,-2.83,47.89,43.70,115436.0,-7023.0,241045.0,-10.89,37.0,X-MC,2.29,72.0,-0.06,1.73,11.79,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,0.01,-11.26,28.08,13.67,64059.0,-28933.0,228130.0,-17.95,32.0,X-LC,2.30,14.0,-0.45,1.64,7.70,XY25,NTT,FMCG
12,BERGEPAINT,680.0,-0.73,-4.49,26.84,21.14,58275.0,-10219.0,217120.0,-11.18,38.0,X-MC,2.38,74.0,-0.18,1.56,22.38,XY24,NTT,PAINTS


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.14,-25.66,120.82,64.15,213718.0,-61061.0,176890.0,-56.31,33.0,X-SC,2.78,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.98,-39.39,123.07,35.20,95979.0,-50683.0,77987.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.07,-0.33,-10.05,42.42,28.11,76854.0,-20235.0,181175.0,-34.87,27.0,X-MC,13.19,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
59,RELAXO,1176.00,0.01,-48.06,193.23,52.31,145690.0,-69763.0,75397.0,-44.73,40.0,X-SC,7.09,91.0,-0.48,0.54,0.92,X40N,NTT,FOOTWEAR
31,HAVELLS,2069.16,-0.50,-10.05,47.28,32.48,139490.0,-32962.0,295029.0,-14.62,40.0,X-MC,6.08,67.0,-0.24,2.12,1.08,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.31,-29.60,102.90,42.84,47091.0,-19242.0,45764.0,-53.72,43.0,X-SC,35.34,83.0,-0.41,0.33,3.54,XY24,NTT,MISC
59,RELAXO,1176.00,0.01,-48.06,193.23,52.31,145690.0,-69763.0,75397.0,-44.73,40.0,X-SC,7.09,91.0,-0.48,0.54,0.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.98,-39.39,123.07,35.20,95979.0,-50683.0,77987.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.02,-4.84,30.39,24.08,36714.0,-6142.0,120808.0,-12.55,32.0,X-SC,5.16,89.0,-0.17,0.87,13.83,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.97,-17.28,73.27,43.32,98709.0,-28152.0,134720.0,-28.92,34.0,X-SC,4.90,90.0,-0.29,0.97,3.44,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.42,-11.51,36.40,20.71,121836.0,-43523.0,334714.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.40,11.84,X40,ATH,IT
41,INFY,1972.00,0.37,8.84,23.34,34.24,79483.0,27656.0,340544.0,-15.29,61.0,X-LC,4.27,2.0,0.35,2.44,17.85,X40,NTT,IT
75,TMPV,600.00,0.97,-28.51,72.02,22.98,116308.0,-64404.0,161494.0,-25.52,35.0,X-LC,6.72,3.0,-0.55,1.16,1.57,XY24,NTT,AUTO
81,VBL,671.64,-0.29,-3.90,41.17,35.67,124962.0,-12314.0,303528.0,-15.30,54.0,X-LC,3.98,4.0,-0.10,2.18,9.42,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.70,3.61,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.29,-16.58,117.77,81.67,92167.0,-15550.0,78260.0,7033.33,46.0,L-SC,12.05,271.0,-0.17,0.56,51.55,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.08,-2.86,25.65,22.05,24412.0,-2805.0,95175.0,-16.44,52.0,H-SC,6.90,164.0,-0.11,0.68,38.34,XR,ATH,FINANCE
13,BSOFT,831.70,1.35,-21.71,92.59,50.79,101962.0,-30531.0,110122.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.22,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.24,-14.47,16.94,0.02,25291.0,-25263.0,149296.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.07,29.86,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.25,-2.21,23.67,20.95,58279.0,-5553.0,246215.0,-3.16,50.0,X-LC,17.15,31.0,-0.10,1.77,32.64,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.35,-21.71,92.59,50.79,101962.0,-30531.0,110122.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.22,XR,ATH,IT
84,WIPRO,420.00,0.64,7.00,61.00,72.27,110596.0,11857.0,181305.0,-8.35,65.0,M-LC,2.61,99.0,0.11,1.30,13.60,XR,NTT,IT
41,INFY,1972.00,0.37,8.84,23.34,34.24,79483.0,27656.0,340544.0,-15.29,61.0,X-LC,4.27,2.0,0.35,2.44,17.85,X40,NTT,IT
1,ABB,7934.00,-0.22,-4.03,51.69,45.57,129775.0,-10555.0,251064.0,-38.36,55.0,H-MC,4.55,120.0,-0.08,1.80,6.83,AR,NTT,ELECTRICAL
73,TCS,4389.97,0.42,-11.51,36.40,20.71,121836.0,-43523.0,334714.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.40,11.84,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.88
2,50,76.69


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.93
MC    30.94
LC    24.15
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.70
X40      23.03
X40N     13.07
XR        9.76
XY25      9.08
AR        9.05
OX40N     7.59
SR        0.99
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.46
H-SC    23.79
X-LC    20.70
M-SC    11.68
X-SC     8.06
H-MC     4.82
L-SC     1.40
M-MC     1.32
M-LC     1.30
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.81,42.48
IT,13.04,-15.63,73.77
FINANCE,9.45,-20.83,73.39
MISC,7.29,-27.87,80.17
ELECTRICAL,6.02,-11.89,52.87
PAINTS,5.84,-13.47,30.33
INSURANCE,4.78,-1.93,36.81
PHARMA,4.13,-1.93,34.02
AUTO,2.83,-32.85,78.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3201986.0,21
AR,1309421.0,10
XR,1301071.0,13
X40,1014198.0,14
X40N,864847.0,9
OX40N,733717.0,10
XY25,376432.0,6
SR,282313.0,2
MH,79113.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3661778.0,25
M-SC,1432645.0,15
X-MC,1418639.0,16
X-LC,841702.0,11
X-SC,797065.0,8
H-MC,406545.0,3
L-SC,264804.0,3
M-LC,110596.0,1
H-LC,75519.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1282105.0      6
           AR         896577.0      5
M-SC       XY24       823886.0      6
H-SC       XR         781441.0      7
X-MC       X40        492849.0      7
           XY24       406961.0      3
H-SC       OX40N      340229.0      4
X-MC       X40N       337814.0      4
X-LC       X40        330776.0      5
X-SC       X40N       308969.0      3
M-SC       OX40N      308425.0      5
X-SC       XY24       297523.0      3
H-SC       SR         282313.0      2
X-LC       X40N       218064.0      2
H-MC       AR         211278.0      2
X-LC       XY24       196244.0      2
H-MC       XY24       195267.0      1
X-SC       X40        190573.0      2
X-MC       XY25       181015.0      2
L-SC       XR         179741.0      2
M-SC       XR         174287.0      2
           AR         126047.0      2
M-LC       XR         110596.0      1
X-LC       XY25        96618.0      2
L-SC       OX40N       85063.0      1
H-SC       MH          79113.0      1
H-LC       AR          75519.0      1
M-MC       XY25        60894.0      1
L-MC       XR          55006.0      1
L-LC       XY25        37905.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
